# NYC-TLC Yellow Trip Metadata Exploration

## Introduction

This notebook explore files metadata of [NYC Taxi and Limousine Commission Yellow Trip Record Data](https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page). It may also be used as a base to inform which yellow trip data files to download and use when perform a specific analysis.

### Data Dictionary

Check [Data Dictionary – Yellow Taxi Trip Records](https://www.nyc.gov/assets/tlc/downloads/pdf/data_dictionary_trip_records_yellow.pdf)

## Extracting the Data

Change `year` to extract (or update) metadata

In [1]:
# !python extract_trips_metadata.py -s web -t yellow -y 2024

## Loading the Data

### Import libraries

In [2]:
import glob
import matplotlib.pyplot as plt
import pyarrow as pa
import pandas as pd

from conf import DATASET_LOCAL_METADATA_PATH

### Load the data

In [3]:
METADATA_FILES = glob.glob(f"{DATASET_LOCAL_METADATA_PATH}/yellow_tripmetadata_*.csv")

In [4]:
df = pd.concat([pd.read_csv(file) for file in METADATA_FILES], ignore_index=True)

### Print data summary

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 185 entries, 0 to 184
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   file_name               185 non-null    object 
 1   file_s3_url             185 non-null    object 
 2   file_cloudfront_url     185 non-null    object 
 3   file_record_type        185 non-null    object 
 4   file_year               185 non-null    int64  
 5   file_month              185 non-null    int64  
 6   file_modification_time  185 non-null    object 
 7   file_num_rows           185 non-null    int64  
 8   file_num_columns        185 non-null    int64  
 9   file_column_names       185 non-null    object 
 10  file_size_bytes         185 non-null    int64  
 11  file_size_mbs           185 non-null    float64
 12  file_size_gbs           185 non-null    float64
 13  file_metadata_source    185 non-null    object 
dtypes: float64(2), int64(5), object(7)
memory 

## Exploring the Data

### What is the total number of all records (rows)?

In [6]:
print("{:,d} records.".format(df["file_num_rows"].sum()))

1,753,861,220 records.


### What is the total compressed size (GBs) of all records?

In [7]:
print("{:,.4f} GBs.".format(df["file_size_gbs"].sum()))

28.5041 GBs.


### Which years are covered by all records?

In [8]:
pd.DataFrame({"file_year": sorted(df["file_year"].unique())})

,file_year
0,2009
1,2010
2,2011
3,2012
4,2013
5,2014
6,2015
7,2016
8,2017
9,2018


### What is the total number of records (rows) per each year?

In [9]:
df2 = df[["file_year", "file_num_rows"]].groupby(by="file_year").sum()
df2 = df2.reset_index()
df2 = df2.sort_values(by="file_num_rows", ascending=False)
df2["file_num_rows"] = df2["file_num_rows"].apply(lambda x: "{:,d}".format(x))
df2

,file_year,file_num_rows
2,2011,"176,887,259"
4,2013,"171,816,340"
3,2012,"171,359,007"
0,2009,"170,896,055"
1,2010,"169,001,162"
5,2014,"165,447,579"
6,2015,"146,039,231"
7,2016,"131,131,805"
8,2017,"113,500,327"
9,2018,"102,871,387"


### What is the total compressed size (GBs) of records per each year?

In [10]:
df3 = df[["file_year", "file_size_gbs"]].groupby(by="file_year").sum()
df3 = df3.reset_index()
df3 = df3.sort_values(by="file_size_gbs", ascending=False)
df3["file_size_gbs"] = df3["file_size_gbs"].apply(lambda x: "{:,.4f}".format(x))
df3

,file_year,file_size_gbs
0,2009,5.3248
1,2010,5.2662
2,2011,2.0566
3,2012,2.0338
5,2014,2.0007
4,2013,1.9967
6,2015,1.8878
7,2016,1.7072
8,2017,1.4852
9,2018,1.3638


### Describe files compressed sizes (MBs)?

In [11]:
df[["file_size_mbs"]].describe().T

,count,mean,std,min,25%,50%,75%,max
file_size_mbs,185.0,157.773896,126.084986,4.236813,55.937411,137.921441,174.708383,516.689856


### Which files have largest compressed sizes (MBs)?

In [12]:
df5 = df[["file_name", "file_size_mbs"]]
df5 = df5.sort_values(by="file_size_mbs", ascending=False)
df5.head(n=10)

,file_name,file_size_mbs
57,yellow_tripdata_2010-05.parquet,516.689856
56,yellow_tripdata_2010-04.parquet,503.829928
74,yellow_tripdata_2009-10.parquet,503.091026
58,yellow_tripdata_2010-06.parquet,493.545179
53,yellow_tripdata_2010-01.parquet,491.526559
59,yellow_tripdata_2010-07.parquet,487.319201
61,yellow_tripdata_2010-09.parquet,484.655313
69,yellow_tripdata_2009-05.parquet,472.375512
76,yellow_tripdata_2009-12.parquet,464.997188
67,yellow_tripdata_2009-03.parquet,460.235812


### Which files have smallest compressed sizes (MBs)?

In [13]:
df6 = df[["file_name", "file_size_mbs"]]
df6 = df6.sort_values(by="file_size_mbs", ascending=True)
df6.head(n=10)

,file_name,file_size_mbs
92,yellow_tripdata_2020-04.parquet,4.236813
93,yellow_tripdata_2020-05.parquet,5.941261
94,yellow_tripdata_2020-06.parquet,9.065016
95,yellow_tripdata_2020-07.parquet,12.767580
96,yellow_tripdata_2020-08.parquet,15.832389
97,yellow_tripdata_2020-09.parquet,20.391405
0,yellow_tripdata_2021-01.parquet,20.681445
1,yellow_tripdata_2021-02.parquet,20.768412
100,yellow_tripdata_2020-12.parquet,21.953617
99,yellow_tripdata_2020-11.parquet,22.490852


### Describe files number of records (rows)?

In [14]:
df[["file_num_rows"]].describe().T

,count,mean,std,min,25%,50%,75%,max
file_num_rows,185.0,9.480331e+06,4.848342e+06,238073.0,3588295.0,10295441.0,13971118.0,16146923.0


### Which files have largest number of records (rows)?

In [15]:
df7 = df[["file_name", "file_num_rows"]]
df7 = df7.sort_values(by="file_num_rows", ascending=False)
df7["file_num_rows"] = df7["file_num_rows"].apply(lambda x: "{:,d}".format(x))
df7.head(n=10)

,file_name,file_num_rows
127,yellow_tripdata_2012-03.parquet,"16,146,923"
151,yellow_tripdata_2011-03.parquet,"16,066,351"
115,yellow_tripdata_2013-03.parquet,"15,749,228"
158,yellow_tripdata_2011-10.parquet,"15,697,804"
74,yellow_tripdata_2009-10.parquet,"15,604,551"
153,yellow_tripdata_2011-05.parquet,"15,554,868"
61,yellow_tripdata_2010-09.parquet,"15,540,209"
57,yellow_tripdata_2010-05.parquet,"15,481,351"
79,yellow_tripdata_2014-03.parquet,"15,428,134"
117,yellow_tripdata_2013-05.parquet,"15,285,052"


### Which files have smallest number of records (rows)?

In [16]:
df8 = df[["file_name", "file_num_rows"]]
df8 = df8.sort_values(by="file_num_rows", ascending=True)
df8["file_num_rows"] = df8["file_num_rows"].apply(lambda x: "{:,d}".format(x))
df8.head(n=10)

,file_name,file_num_rows
92,yellow_tripdata_2020-04.parquet,"238,073"
93,yellow_tripdata_2020-05.parquet,"348,415"
94,yellow_tripdata_2020-06.parquet,"549,797"
95,yellow_tripdata_2020-07.parquet,"800,412"
96,yellow_tripdata_2020-08.parquet,"1,007,286"
97,yellow_tripdata_2020-09.parquet,"1,341,017"
0,yellow_tripdata_2021-01.parquet,"1,369,769"
1,yellow_tripdata_2021-02.parquet,"1,371,709"
100,yellow_tripdata_2020-12.parquet,"1,461,898"
99,yellow_tripdata_2020-11.parquet,"1,509,000"


### How does column names change in files?

In [17]:
df9 = df[["file_year", "file_column_names"]].groupby(by=["file_year", "file_column_names"]).size()
df9 = df9.reset_index(name="num_of_files")
pd.set_option('display.max_colwidth', None)
df9

,file_year,file_column_names,num_of_files
0,2009,"vendor_name,Trip_Pickup_DateTime,Trip_Dropoff_DateTime,Passenger_Count,Trip_Distance,Start_Lon,Start_Lat,Rate_Code,store_and_forward,End_Lon,End_Lat,Payment_Type,Fare_Amt,surcharge,mta_tax,Tip_Amt,Tolls_Amt,Total_Amt",12
1,2010,"vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,rate_code,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount",10
2,2010,"vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,rate_code,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount,__index_level_0__",2
3,2011,"VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee",12
4,2012,"VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee",12
5,2013,"VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee",12
6,2014,"VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee",12
7,2015,"VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee",12
8,2016,"VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee",12
9,2017,"VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee",12


### How many times a column name appear in files?

In [18]:
df10 = df["file_column_names"].str.split(',').explode()
df10 = pd.DataFrame(df10)
df10 = df10.groupby(by='file_column_names').size()
df10 = df10.reset_index(name="num_of_files")
df10 = df10.sort_values(by="num_of_files", ascending=False)
df10

,file_column_names,num_of_files
28,mta_tax,185
26,fare_amount,173
40,total_amount,173
29,passenger_count,173
36,store_and_fwd_flag,173
39,tolls_amount,173
38,tip_amount,173
43,trip_distance,173
30,payment_type,173
41,tpep_dropoff_datetime,161


### Which files have longitude and latitude?

In [19]:
df11 = df[(df["file_column_names"].str.contains("long", case=False) | 
           df["file_column_names"].str.contains("lat", case=False))]
df11 = df11[["file_size_mbs", "file_cloudfront_url"]]
pd.set_option('display.max_colwidth', None)
df11

,file_size_mbs,file_cloudfront_url
53,491.526559,https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2010-01.parquet
54,342.101726,https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2010-02.parquet
55,389.782040,https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2010-03.parquet
56,503.829928,https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2010-04.parquet
57,516.689856,https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2010-05.parquet
58,493.545179,https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2010-06.parquet
59,487.319201,https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2010-07.parquet
60,378.744400,https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2010-08.parquet
61,484.655313,https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2010-09.parquet
62,452.643692,https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2010-10.parquet
